In [3]:
# IMPORTS and VARIABLES
import pandas as pd
import numpy as np
import json
import geopandas as gp
import h3
from placekey.api import PlacekeyAPI
from keplergl import KeplerGl
import pprint

from glob import glob
# https://stackoverflow.com/a/65001152/12572612: from multiprocessing import Pool
from multiprocess import Pool

placekey_api_key = "NDd0VsiPKznvm89NLNj9DHrtU3GVzZ1h" 
pk_api = PlacekeyAPI(placekey_api_key)

# 50 states + DC + 6 territories (PR, VI, AS, GU, MP, UM)
SF57 = {'01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE', '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI', '56': 'WY', '60': 'AS', '66': 'GU', '69': 'MP', '72': 'PR', '74': 'UM', '78': 'VI'}
SF57R = {value : key for (key, value) in SF57.items()}
# 50 states + DC + PR + VI
SF53 = {'01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE', '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI', '56': 'WY', '72': 'PR', '78': 'VI'}
SF53R = {value : key for (key, value) in SF53.items()}
# 50 states + DC + PR
SF52 = {'01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE', '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI', '56': 'WY', '72': 'PR'}
SF52R = {value : key for (key, value) in SF52.items()}
# 50 states + DC
SF51 = {'01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE', '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI', '56': 'WY'}
SF51R = {value : key for (key, value) in SF51.items()}



/home/nhat/anaconda3/envs/py39/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.0-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


# Read all attom raw

# Method 1: read and filter raw attom .txt. 30min to get MO state

In [17]:
# # for all 16 attom raw txt, take sec to lazy load!
# # ParserError: Error tokenizing data. C error: EOF inside string starting at row 49533
# # ERROR: Window attom EOL encoding error: encoding_errors replace to fix!

# can subset columns instantly!
# ATTOM DATA:  Our files are ANSI-1252 encoding (cp1252)
# https://stackoverflow.com/a/29857126/12572612
# EOF error in dask: https://github.com/dask/dask/issues/4145
all_attom = dd.read_csv('attom/data/*.txt', dtype=str, sep='\t', header = 0,
     usecols = attom_cols + ['SitusStateCode'], # select columns to load, avoid out of mem
     # engine='python', # TOO SLOW!
                        encoding='cp1252', 
                        # error_bad_lines=False, # deprecated
                        quoting=3, # probably the culprit * 
    encoding_errors="replace", # need this to deal with Attom Window EOL chars
                       )
# ((Delayed('int-e501ced1-a693-418c-8d47-6a619f0a188d'), 19), 2748)
print(all_attom.shape, all_attom.npartitions)

# Filter by state and output to csv/hdf5
# MO filter: 30 min!
# same error: at min5: ParserError: Error tokenizing data. C error: EOF inside string starting at row 49533  
mo_attom = all_attom.loc[all_attom.SitusStateCode == "MO"]
# Repartition to reduce overhead
mo_attom = mo_attom.repartition(npartitions = mo_attom.npartitions // 50)
mo_attom = mo_attom.compute() # instant lazy filter
mo_attom.shape # (3330480, 19)

(Delayed('int-d3311e27-867d-4d18-b905-e7a5d95a83ab'), 19) 2748


(3330480, 19)

time: 28min 10s (started: 2022-04-28 15:07:14 -04:00)


# Method 2: 
- Parse raw attom to county .csv files (all potentially useful fields)
- Dask/Pool read_csv

In [4]:
use_cols = ['[ATTOM ID]', 'AreaBuilding', 'AreaBuildingDefinitionCode', 'AreaLotSF', 'AssessorLastSaleAmount', 'AssessorLastSaleDate', 'AssessorPriorSaleAmount', 'AssessorPriorSaleDate', 'BedroomsCount', 'BuildingsCount', 'CensusBlock', 'CompanyFlag', 'ContactOwnerMailAddressFull', 'ContactOwnerMailAddressInfoFormat', 'DeedOwner1NameFull', 'Fireplace', 'FireplaceCount', 'GeoQuality', 'LegalDescription', 'OwnerTypeDescription1', 'ParcelNumberRaw', 'ParkingGarage', 'ParkingGarageArea', 'ParkingSpaceCount', 'PartyOwner1NameFull', 'PartyOwner2NameFull', 'PartyOwner3NameFull', 'PreviousAssessedValue', 'PropertyAddressCity', 'PropertyAddressFull', 'PropertyAddressHouseNumber', 'PropertyAddressState', 'PropertyAddressStreetDirection', 'PropertyAddressStreetName', 'PropertyAddressStreetPostDirection', 'PropertyAddressStreetSuffix', 'PropertyAddressUnitPrefix', 'PropertyAddressUnitValue', 'PropertyAddressZIP', 'PropertyAddressZIP4', 'PropertyLatitude', 'PropertyLongitude', 'PropertyUseGroup', 'PropertyUseStandardized', 'RoofConstruction', 'RoomsCount', 'RoomsFamilyCode', 'RoomsGameFlag', 'SitusCounty', 'SitusStateCode', 'StatusOwnerOccupiedFlag', 'StoriesCount', 'TaxAssessedValueImprovements', 'TaxAssessedValueTotal', 'TaxMarketValueImprovements', 'TaxMarketValueTotal', 'TaxMarketValueYear', 'TaxYearAssessed', 'UnitsCount', 'YearBuilt']
SF57 = {'01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE', '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI', '56': 'WY', '60': 'AS', '66': 'GU', '69': 'MP', '72': 'PR', '74': 'UM', '78': 'VI'}
SF57R = {value : key for (key, value) in SF57.items()}
# what if someone names Null, or None, or Nan?
possible_bad_values = {"", " ", "Null", "None", "NONE", "none"}

# # ALL 16 ATTOM txt files has same header
headers = []
for num in range(1,17):
    num = str(num).zfill(2)
    with open(f'attom/data/{num}.txt', 'r', errors='replace') as f:
        header = tuple(next(f).strip().split('\t'))
        headers.append(header)
# Checked!        
len(set(headers)) # just 1 header ordering!
HEADER = set(headers).pop()
HEADER_DICT = {col_name : idx for idx, col_name in enumerate(HEADER)}

## Parse: RUN ONCE

In [107]:
# ! rm attom/data/attom/*

rm: cannot remove 'attom/data/attom/*': No such file or directory
time: 310 ms (started: 2022-04-28 20:55:29 -04:00)


### Atom

In [108]:
# import csv
# def attom_io(input_file):
#     with open(input_file, errors='replace', encoding='cp1252',) as readfile:
#         zip_order = input_file.split('/')[-1][:2] # e.g. 03, or 11
#         next(readfile)
#         position = 0
#         use_idx = [HEADER_DICT[col_name] for col_name in use_cols]
#         for line in readfile:
#             position += 1
#             # Strip invalid EOL chars here is important!
#             line = line.strip().split('\t')
#             sf_code = line[HEADER_DICT['SitusStateCode']]  
            
#             if not sf_code:
#                 print(f"ALERT: null statecode file@row= {input_file}@{position}")
#             elif sf_code not in SF57R:
#                 print(f"ALERT: statecode outside SF57R, {sf_code}, file@row= {input_file}@{position}")
#             else: 
#                 # WRITE to individual state df
#                 # output file: county, e.g. AK_DENALI.json
#                 county = line[HEADER_DICT['SitusCounty']]
#                 county = "".join(county.split())
#                 stem = (sf_code + '_' + county).lower()                    
#                 ofile = f'attom/data/attom/{stem}.csv'

#                 # select columns; and add addresses, and source locations (zip file, line positions)
#                 line = [str(line[idx]).strip() for idx in use_idx] 
#                 # All other non-string data are stringified with str() before being written
#                 line = ["" if value in possible_bad_values 
#                         else value for value in line] + [zip_order, position]
                
#                 # IMPORTANT: use csv.writer to deal with csv quoting and newlines
#                 # https://stackoverflow.com/questions/3191528/csv-in-python-adding-an-extra-carriage-return-on-windows
#                 # https://stackoverflow.com/questions/43733205/write-newline-to-csv-in-python
#                 with open(ofile, 'a', newline='') as csvfile:
#                     # pd.read_csv(quoting=0 QUOTE_MINIMAL)
#                     writer = csv.writer(csvfile, delimiter='\t',
#                                         quoting=csv.QUOTE_MINIMAL)
#                     writer.writerow(line)  
#                     # writefile.write("\t".join(line) + '\n')                           

# # 12 min to write to all county files     
# # If Pool(processes) is None then the number returned by os.cpu_count() is used.
# with Pool() as pool:
#     pool.map(attom_io, glob('attom/data/*.txt'))

# "Complete parsing"

'Complete parsing'

time: 14min 33s (started: 2022-04-28 20:55:45 -04:00)


## Parallel read with dask / Pool

In [5]:
import dask.dataframe as dd

attom_cols = ['[ATTOM ID]',     
    'LegalDescription', 'OwnerTypeDescription1', 'ParcelNumberRaw', 
    'PartyOwner1NameFull', 'PartyOwner2NameFull', 
    'PropertyAddressCity', 
    'PropertyAddressFull', 
    'PropertyAddressHouseNumber', 'PropertyAddressStreetDirection', 'PropertyAddressStreetName', 
    'PropertyAddressStreetPostDirection', 'PropertyAddressStreetSuffix', 'PropertyAddressUnitPrefix', 'PropertyAddressUnitValue', 
    'PropertyAddressZIP', 
    'PropertyLatitude', 'PropertyLongitude',]


sf = "mo"

# dask vs Pool pd.read_csv: 
# MO: ~same 40sec
# FL: 50sec <> 25sec; TX 1min <> 1.5min

# # instant lazy load with dask; provide header by names= (dont need header at row0)
# df = dd.read_csv(f'attom/data/attom/{sf}*.csv', dtype=str, sep='\t',
#                  names = use_cols + ['zorder', 'rorder'], # added 2 columns
#                  usecols = attom_cols, # can select columns to load to mem with usecols=
#                 )
# print(df.npartitions)

# # Convert dask df to pandas df: 15sec for MO! 
# df = df.compute()

# parsing above: encoding (with <> without), handle errors (replace <> ignore)
# # ??? Why the difference?? vs previous json = 3.330.480 
# # read_csv could have skip all nan rows for selected usecols; but all rows has ATTOM ID
# df.shape # MO = 3.329.589 ( -891) ; FL = (10_500_171, 18)


def pool_read_csv(filepath):
    return pd.read_csv(filepath, dtype=str, sep='\t',
                 names = use_cols + ['zorder', 'rorder'], # added 2 columns
                 usecols = attom_cols, # can select columns to load to mem with usecols=
                )
with Pool() as pool:
    attom = pool.map(pool_read_csv, glob(f'attom/data/attom/{sf}*.csv'))
    attom = pd.concat(attom)
attom.shape    # MO SHOULD BE: 25sec, (3330480, 19)

(3330480, 18)

In [277]:
x = attom.query("Address == Address & PropertyAddressStreetName != PropertyAddressStreetName")[
    addr_cols].drop(columns=['Address1', 'Address2'])
x.columns = ['a', 'Address', 'houseNumber', 'streetDir', 'streetName',
            'streetPostDir', 'streetSuffix','unitPrefix', "unitValue"]
x

,a,Address,houseNumber,streetDir,streetName,streetPostDir,streetSuffix,unitPrefix,unitValue
13268,204181467,14610 NW 73RD ST,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13271,198083407,12703 N ROBINHOOD LN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15480,202559166,7695 N EVERTON AVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15505,204956090,6105 NW 103RD ST,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16806,186648433,6100 NW PLATTE DR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
14727,300329574,TALLEY BEND & 2025 RD,NaN,NaN,NaN,NaN,NaN,NaN,& 2025
1338,299658133,N SALT CREEK SLOUGH & S OF ST RT Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237,288003202,KING S BRANCH & E AVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
442,288005623,CARPENTER,NaN,NaN,NaN,NaN,NaN,CARPENTER,NaN


## assemble Address1 and Address2 

In [ ]:
def addr_empty(*args):
    args = [str(i).strip() for i in args]
    return " ".join(i for i in args if (i and i != 'nan'))
addr_empty_vectorize = np.vectorize(addr_empty)

attom['Address1'] = addr_empty_vectorize(attom.PropertyAddressHouseNumber,
attom.PropertyAddressStreetDirection,
attom.PropertyAddressStreetName,
attom.PropertyAddressStreetSuffix,
attom.PropertyAddressStreetPostDirection)

attom['Address2'] = addr_empty_vectorize(attom.PropertyAddressUnitPrefix,
attom.PropertyAddressUnitValue,)


# Normalize/Clean data

In [7]:
# keep only alphanumeric
keepchars = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
del_chars = ''.join(c for c in map(chr, range(1114111)) if not c in set(keepchars))
# KEEP THE UPPERCASE VERSION
keepchars_table = str.maketrans(keepchars, keepchars.upper(), del_chars)


In [8]:
attom = attom.rename(columns= {
    '[ATTOM ID]': 'aid',
    'PropertyAddressFull' : 'Address',
    'LegalDescription' : 'Legal', 
    'ParcelNumberRaw' : 'Numb', 
    'PartyOwner1NameFull' : "Owner",
    'PartyOwner2NameFull' : 'Owner2',
    'PropertyAddressCity' : "ACity",
    'PropertyAddressZIP' : 'AZip',
    'PropertyLatitude' : 'Lat',
    'PropertyLongitude' : 'Lon', 
})

for col in ['Legal', 'Numb']:
    attom[col] = attom[col].str.translate(keepchars_table)

In [9]:
# add normalized address fields: dont need
# attom['AddressC'] = attom['Address'].str.translate(keepchars_table)
# attom['Address1C'] = attom['Address1'].str.translate(keepchars_table)
# attom['Address2C'] = attom['Address2'].str.translate(keepchars_table)

# REPLACE BAD VALUES
for bad_value in ["", " ", "NONE", "None"]:
    attom.replace(bad_value, np.nan, inplace = True)
    
# # replace python NoneType with np.nan
attom.fillna(np.nan, inplace=True) 


In [10]:
cols = ["Lat", "Lon"]
attom[cols] = attom[cols].astype(float)
attom.shape # 11sec (3.330.480, 41)

(3330480, 20)

In [13]:
addr_cols = [
    'aid',
    'Address',
 'Address1', 'Address2',
     'PropertyAddressHouseNumber',
 'PropertyAddressStreetDirection',
 'PropertyAddressStreetName',
 'PropertyAddressStreetPostDirection',
 'PropertyAddressStreetSuffix',
 'PropertyAddressUnitPrefix',
 'PropertyAddressUnitValue',
]

In [11]:
# https://stackoverflow.com/a/54099389/12572612 
# .query: need engine="python" with non-NaN native dtype
# https://stackoverflow.com/a/51735886/12572612
# NOT NULL: df.query("value == value") ;; NULL: df.query("value != value")
attom.query("Address.notna() & Address1.isna()", engine='python')[cols]
attom.query("Address == Address & Address1 != Address1")[cols].shape # 572


(572, 2)

In [ ]:
# If Address1 null, set addr = address1
attom['Address1'] = np.where(
    attom.Address.notna() & attom.Address1.isna(), 
    attom['Address'], attom['Address1'])
# If addr1 = addr, set addr2 = empty
attom['Address2'] = np.where(
    attom.Address == attom.Address1, 
    np.nan, attom['Address2'])

In [14]:
# Sanity check: when Address is null, no other addr subfields exist!
attom.query("Address != Address")[addr_cols].count()

aid                                   508206
Address                                    0
Address1                                   0
Address2                                   0
PropertyAddressHouseNumber                 0
PropertyAddressStreetDirection             0
PropertyAddressStreetName                  0
PropertyAddressStreetPostDirection         0
PropertyAddressStreetSuffix                0
PropertyAddressUnitPrefix                  0
PropertyAddressUnitValue                   0
dtype: int64

### Conclusion: you get what [data] you paid for
- ATTOM cheap = data full of holes
    - has Addr, but all Addr subfields are null! e.g. aid 204181467, 198083407
- 16 unorganized txt files
- new changes update email update at all?
- bad support, no doc, not even trying to deposit to our google bucket
- 

### ATTOM: Successfully split Addr = Addr1 + Addr2

In [19]:
attom[['aid', 'Address', 'Address1']].nunique()

aid         3330480
Address     2409972
Address1    2356977
dtype: int64

In [127]:
attom_pk_maps = {
    'aid' : 'query_id', # unique row id
    # https://www.placekey.io/tutorials/using-placekey-without-addresses-latitude-and-longitude-only
    'Lat' : 'latitude', # do need these fields, in case missing address field
    'Lon' : 'longitude', # do need these fields, in case missing address field
    'Address1' : 'street_address',
    'ACity' : 'city',
    'AZip' : 'postal_code',
}

def gen_placekey_df(df, maps):
    keep_cols = list(maps.values())
    # only need placekey cols
    # before renaming, drop columns with names similar to Placekey required columns
    df = df.drop(columns=keep_cols, 
                 errors='ignore').rename(columns=maps)[keep_cols]

    # df['location_name'] = 'attom_null'
    
#     # coerce types to conform to Placekey API
#     df[['latitude', 'longitude']] = df[['latitude', 'longitude']].astype(float)
#     df[['query_id', 'street_address', 'city']] = df[['query_id', 'street_address', 'city']].astype(str)
    
#     # zipcode = 00000 means null; ATTOM zipcode could be a float, or None, ""
#     # so, need a placeholder value to convert to int, zfill, back to np.nan
#     NAN_ZIPS = "00000"
#     df['postal_code'] = df['postal_code'].fillna(NAN_ZIPS).astype(int).astype(str).str.zfill(5).replace(NAN_ZIPS, np.nan)
    return df

In [113]:
attom_addr2 = attom.query("Address2 == Address2").copy()
attom.shape, attom_addr2.shape

((3330480, 20), (90651, 20))

In [128]:
pk_attom = gen_placekey_df(attom_addr2, attom_pk_maps)
# possible_bad_values = ["", " ", "null", "Null", "None", "nan", "Nan"]  
# for bad_value in possible_bad_values:
#     pk_attom = pk_attom.replace(bad_value, np.nan)
# # replace NoneType with np.nan    
# pk_attom.fillna(np.nan, inplace=True)    
# ((517596, 8)
pk_attom.shape # (3330480, 6)

(90651, 6)

In [84]:
def pk_lookup(df, sf_code):
    # add missing hard-coded columns (str type)
    df['iso_country_code'] = 'US' 
    df['region'] = sf_code     
    df = json.loads(df.to_json(orient='records'))
    # Rate limit: 100 bulk req per min x 100 addrs per bulk req
    # i.e. 10,000 addr per min 
    responses = pk_api.lookup_placekeys(df, 
                                    strict_address_match=False,
                                    strict_name_match=False, 
                                    # verbose=True,
                                   )
    
    def clean_api_responses(data_jsoned, responses):
        print("number of requests sent: ", len(data_jsoned))
        print("total queries returned:", len(responses))
        # filter out invalid responses
        responses_cleaned = [resp for resp in responses if 'query_id' in resp]
        print("total successful responses:", len(responses_cleaned))
        return responses_cleaned   
    
    responses = clean_api_responses(df, responses)
    return pd.read_json(json.dumps(responses), dtype={'query_id':str})

# # TEST single placekey lookup!
# def load_json(id):
#     query = json.loads(
#     pk_attom.query("query_id == @id").to_json(orient="records"))[0]
#     return query

# all no pk-what!
# single_place_example = load_json('239420438') # 'placekey': '@5pt-cjv-ht9'}
# # single_place_example = load_json('210392065') # 'placekey': '@5r8-28d-w49'}
# print(single_place_example)
# pk_api.lookup_placekey(**single_place_example)

### API CALL: Address v.s. Address1

In [276]:
pk_attom.shape

(90651, 8)

In [130]:
sf_code = 'MO'
# Called once with Address, now with Address1
pked_attom = pk_lookup(pk_attom, sf_code)
pked_attom.placekey.count()

number of requests sent:  90651
total queries returned: 90651
total successful responses: 90651


64762

In [133]:
# Address1: Invalid address                    22089
pked_attom.error.value_counts()

Invalid address                    22089
All queries in batch had errors     3800
Name: error, dtype: int64

In [135]:
# # RETRY API error responses
# # another error: Missing placekeys due to exceeding API rate limit
# pk_errors = pked_attom[pked_attom.error == 
#                                "All queries in batch had errors"].query_id
# pk_errors = pk_attom[pk_attom.query_id.isin(pk_errors)].copy()

# pk_errors = pk_lookup(pk_errors, sf_code)
# pk_errors.error.value_counts() # All queries in batch had errors    3900
# # If pk_errors gives new results, pd.update pked_attom

In [137]:
pk_attom_joined = pd.merge(
    attom, pked_attom, left_on='aid', 
    right_on="query_id", how='left').drop(columns='query_id')

In [138]:
# pk_attom_joined_address = pk_attom_joined.copy()
# pk_attom_joined_address1 = pk_attom_joined.copy()

In [160]:
# pk_attom_joined = pk_attom_joined_address.copy()
# pk_attom_joined1 = pk_attom_joined_address1.copy()



In [161]:
# pk api requests using address
test = pk_attom_joined.query("placekey == placekey")[['aid', 'Address', 'Address1', 'Address2', 'placekey']]
test[['pkwhat', 'pkwhere']] = test.placekey.str.split("@", expand=True)
test.replace("", np.nan, inplace=True)
# pk api requests using address1
test1 = pk_attom_joined1.query("placekey == placekey")[['aid', 'Address', 'Address1', 'Address2', 'placekey']]
test1[['pkwhat', 'pkwhere']] = test1.placekey.str.split("@", expand=True)
test1.replace("", np.nan, inplace=True)

# # Only very few address with Address2 receive pkwhat
# test.query('pkwhat == pkwhat & Address2==Address2').shape, 

In [175]:
# All successful API responses are (SubAddress-full), and wh-full (contain pkwhat, and pkwhere)
pd.concat([test.count(), test1.count()], axis=1)

,0,1
aid,63945,64762
Address,63945,64762
Address1,63945,64762
Address2,63945,64762
placekey,63945,64762
pkwhat,63945,64762
pkwhere,63945,64762


In [195]:
# IMPORTANT: CHECK THE DIFFERENCE
# different Address2 cause Address-PK to have more placekeys than Address1-PK
print(test.placekey.nunique(), test1.placekey.nunique())

56399 25509


In [224]:
# diff and same placekeys: for Address <> Address1
test_merged = test[['aid', 'Address', 'Address1', 'placekey']].merge(
    test1[['aid', 'Address1', 'Address', 'placekey']], 
    on='aid', how='outer', suffixes=('', '_y'))
print(test_merged.shape)
test_merged.iloc[np.r_[0:3, -3:0]]

(64807, 7)


,aid,Address,Address1,placekey,Address1_y,Address_y,placekey_y
0,299881888,72562 MINERAL ST # 72536,72562 MINERAL ST,22b@5pp-kjc-9cq,72562 MINERAL ST,72562 MINERAL ST # 72536,22b@5pp-kjc-9cq
1,299876150,819 SCHROEDER DR # 821,819 SCHROEDER DR,22g@5pp-kkb-5xq,819 SCHROEDER DR,819 SCHROEDER DR # 821,22g@5pp-kkb-5xq
2,299879501,107 S BRETT DR # B,107 S BRETT DR,22s@5pq-vg6-kxq,107 S BRETT DR,107 S BRETT DR # B,223@5pq-vg6-kxq
64804,299653374,NaN,NaN,NaN,3882 HWY 5 & 240 3842,3882 HWY 5 & 240 & 3842 HWY 5 & 240,222@5pq-jvd-xkf
64805,299656541,NaN,NaN,NaN,198 HWY 5 & 240 196,198 HWY 5 & 240 & 196 HWY 5 & 240,224@5pq-jyk-m6k
64806,288008830,NaN,NaN,NaN,22994 OUTER RD,22994 OUTER RD SCAMMAN I,222@5nz-bpz-rkz


In [230]:
## Address1 only generates new placekeys! # (862, 7)
test_merged.query("aid == aid & placekey != placekey").sample(8)

,aid,Address,Address1,placekey,Address1_y,Address_y,placekey_y
64082,31368775,NaN,NaN,NaN,1720-1724 CHOUTEAU AVE,1720-1724 CHOUTEAU AVE # 208,224@5pj-5hs-sbk
64422,202092934,NaN,NaN,NaN,4744A MCPHERSON AVE,4744A MCPHERSON AVE # 204,23d@5pj-5hf-snq
64805,299656541,NaN,NaN,NaN,198 HWY 5 & 240 196,198 HWY 5 & 240 & 196 HWY 5 & 240,224@5pq-jyk-m6k
64118,197377150,NaN,NaN,NaN,3212-3214 LOCUST ST,3212-3214 LOCUST ST # 3214,24b@5pj-5h7-cqz
64243,194687277,NaN,NaN,NaN,4137-4139 CASTLEMAN AVE,4137-4139 CASTLEMAN AVE # 2E,229@5pj-5h9-snq
64603,202298110,NaN,NaN,NaN,5316-5320 PERSHING AVE,5316-5320 PERSHING AVE # 205,224@5pj-5h8-vmk
64282,253481663,NaN,NaN,NaN,1113-1115 LOCUST ST,1113-1115 LOCUST ST # 400,226@5pj-5gc-rhq
64578,205590004,NaN,NaN,NaN,1219-1221 WASHINGTON AVE,1219-1221 WASHINGTON AVE # 300,223@5pj-5gc-rx5


In [229]:
## CAN SAFELY IGNORE THESE few BAD ADDRESSES!
## Address only generates new placekeys! (45,)
test_merged.query("aid == aid & placekey_y != placekey_y").sample(3)

,aid,Address,Address1,placekey,Address1_y,Address_y,placekey_y
38071,214969786,14568 COUNTY LN 250,14568 COUNTY LN,222@5r7-yt3-sdv,NaN,NaN,NaN
37919,240319619,25200 HALLMARK LN # 25208,25200 HALLMARK LN,225@5r8-2c6-5s5,NaN,NaN,NaN
38011,324207986,380 COUNTY LN 130,380 COUNTY LN,222@5r7-zpy-68v,NaN,NaN,NaN


In [ ]:
### GROUPBY check address

In [244]:
testgrouped = test_merged.groupby('placekey').agg(lambda x: list(x))
testgrouped['count'] = test_merged.groupby('placekey').size()

testgroupedmul = testgrouped.query("count > 1").sort_values('count')
print(testgroupedmul.shape)
testgroupedmul.tail(2)['count']

(1633, 7)


placekey
22f@5pq-zfs-7dv    235
225@5pj-5gc-sh5    258
Name: count, dtype: int64

In [245]:
testgrouped = test_merged.groupby('placekey_y').agg(lambda x: list(x))
testgrouped['count'] = test_merged.groupby('placekey_y').size()

testgroupedmul = testgrouped.query("count > 1").sort_values('count')
print(testgroupedmul.shape)
testgroupedmul.tail(2)['count']

(5424, 7)


placekey_y
22f@5pq-zfs-7dv    236
225@5pj-5gc-sh5    307
Name: count, dtype: int64

In [265]:
testgrouped = test.groupby('placekey').agg(lambda x: list(x))
testgrouped['count'] = test.groupby('placekey').size()
testgrouped = testgrouped.query("count > 1").sort_values('count')
print(testgrouped['count'].sum())

9179


In [267]:
testgrouped1 = test1.groupby('placekey').agg(lambda x: list(x))
testgrouped1['count'] = test1.groupby('placekey').size()
testgrouped1 = testgrouped1.query("count > 1").sort_values('count')
print(testgrouped1['count'].sum())

44677


## OUTCOME
- 9179 Address with mapped placekeys
- 44_677 Address1 with mapped placekeys

In [257]:
len(set(test.aid) - set(test1.aid)), len(set(test1.aid) - set(test.aid))

(45, 862)

In [ ]:
### Conclusion:
- Requests: 90_651  Address (Address2 attached) / Address1 (Address2 separated)
- Resuts: 63_945 / 64_762 
- pkwhat: 63_945 / 64_762
- unique placekey: 56_399 / 25_509 (emphasis)

In [ ]:
# same address1, diff lat and lon, or diff city?
# that's the whole point of using placekey
# Do not drop/duplicated street_address, or anything, before querying!

In [51]:
pk_attom

,query_id,latitude,longitude,street_address,city,postal_code
0,318261730,38.632752,-92.583055,NaN,NaN,NaN
1,312215716,38.766883,-92.467466,21087 HIGHWAY 179,JAMESTOWN,65046
2,312215790,38.624914,-92.575849,NaN,CALIFORNIA,65018
3,312215774,38.690601,-92.577729,26387 HIGHWAY D,CALIFORNIA,65018
4,325040788,38.609600,-92.620956,31820 FLAG SPRING RD,CALIFORNIA,65018
...,...,...,...,...,...,...
5706,322439862,40.440865,-95.235704,19703 X AVE,TARKIO,64491
5707,320978791,40.526107,-95.355229,13919 R,WESTBORO,64498
5708,320978833,40.372588,-95.197870,24451 Z AVE,FAIRFAX,64446
5709,320978792,40.485208,-95.309406,NaN,TARKIO,64491


## ATTOM no coords (without addresses) = hopeless

## ATTOM no coords (with addresses): Placekey-ed will handle the standardizing and geocoding!

## ATTOM: coords (without addresses): Placekey-ed also handles this!
- https://www.placekey.io/tutorials/using-placekey-without-addresses-latitude-and-longitude-only


In [ ]:
# TODO: Regrid: replace with Nan:
NO ADDRESS ASSIGNED BY CITY
No Situs Address
RURAL

In [279]:
df = gp.read_file('/home/nhat/regrid-bucket/ak_denali.geojson.zip', ignore_geometry = True)
df.shape

(3127, 128)

In [288]:
cols = ['address', 'saddno', 'saddpref', 'saddstr', 'address2', 'sunit', ]
df.query('address == address & saddno != saddno')[cols]

,address,saddno,saddpref,saddstr,address2,sunit
23,PARK LN TRACT F-2-B,None,None,PARK LN TRACT F-2-B,None,# F-2-B
29,HILLTOP RD,None,None,HILLTOP RD,None,None
41,SULFIDE WY,None,None,SULFIDE WY,& SPUR RD ON THE CORNER,None
252,PARKS & DENALI HWY,None,None,PARKS & DENALI,None,None
262,S SYLVAN RD,None,S,SYLVAN,LIVES AT END NO ST NAME,None
...,...,...,...,...,...,...
3106,STIBNITE RD,None,None,STIBNITE RD,None,None
3112,QUARTZ WY,None,None,QUARTZ WY,None,None
3113,STIBNITE RD,None,None,STIBNITE RD,None,None
3121,CHRISTIANSEN LAKE SUBD,None,None,CHRISTIANSEN LAKE SUBD,BLOCK 1 LOT 7,None


# REGRID address1


# should have same result as ATTOM